# Operar no Meta Trader usando modelo

## Usando o modelo do WebScraping_Fundamentus:

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = 'https://www.fundamentus.com.br/resultado.php'

driver.get(url)
local_tabela = '/html/body/div[1]/div[2]/table'
elemento = driver.find_element("xpath", local_tabela)
html_tabela = elemento.get_attribute('outerHTML')
tabela = pd.read_html(str(html_tabela), thousands = '.', decimal = ',')[0]

tabela = tabela.set_index("Papel")
tabela = tabela[['Cotação', 'EV/EBIT', 'ROIC', 'Liq.2meses']]
tabela['ROIC'] = tabela['ROIC'].str.replace("%", "")
tabela['ROIC'] = tabela['ROIC'].str.replace(".", "")
tabela['ROIC'] = tabela['ROIC'].str.replace(",", ".")
tabela['ROIC'] = tabela['ROIC'].astype(float)

tabela = tabela[tabela['Liq.2meses'] > 1000000]
tabela = tabela[tabela['EV/EBIT'] > 0]
tabela = tabela[tabela['ROIC'] > 0]

tabela['ranking_ev_ebit'] = tabela['EV/EBIT'].rank(ascending = True)
tabela['ranking_roic'] = tabela['ROIC'].rank(ascending = False)
tabela['ranking_total'] = tabela['ranking_ev_ebit'] + tabela['ranking_roic']

tabela = tabela.sort_values('ranking_total')

C:\Users\dudiz\AppData\Local\Temp\ipykernel_9420\1947053643.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tabela['ROIC'] = tabela['ROIC'].str.replace(".", "")


In [4]:
# Seleciona as 10 principais entradas do DataFrame 'tabela'
tabela = tabela.head(10)

# Obtém os rótulos (tickers) correspondentes a essas 10 principais entradas
tickers = tabela.index

# Exibe os tickers
tickers

Index(['PSSA3', 'PETR4', 'PETR3', 'UNIP6', 'KEPL3', 'WIZC3', 'VLID3', 'GOAU4',
       'PLPL3', 'FESA4'],
      dtype='object', name='Papel')

## Baixa e configura o MT5, cada corretora configura de uma forma

In [5]:
!pip install MetaTrader5

     ---------------------------------------- 0.0/49.9 kB ? eta -:--:--
     -------------------------------- ------- 41.0/49.9 kB ? eta -:--:--
     -------------------------------------- 49.9/49.9 kB 842.9 kB/s eta 0:00:00


Você está instalando o pacote **MetaTrader5**, que é uma biblioteca Python que **permite a integração com a plataforma de negociação MetaTrader 5**. Esta biblioteca pode ser útil para automatizar operações de negociação e análise de dados no MetaTrader 5.

In [6]:
import MetaTrader5 as mt5

In [7]:
mt5.initialize()

False

O comando **mt5.initialize()** inicializa a conexão com a plataforma MetaTrader 5, permitindo que você se comunique e execute operações a partir do Python. Essa inicialização é necessária antes de usar qualquer funcionalidade da API do MetaTrader 5.

### Documentação do MT5 

https://www.mql5.com/en/docs/python_metatrader5

## Dicionário da ordem de compra

In [8]:
# Itera sobre os tickers disponíveis na lista 'tickers'
for ticker in tickers:

    # Seleciona o símbolo (ativo) para o qual a ordem será enviada
    mt5.symbol_select(ticker)
    
    # Obtém o preço de venda atual do ativo
    preco = mt5.symbol_info(ticker).ask
    
    # Define a quantidade a ser comprada (neste caso, 100 unidades)
    quantidade = 100.0
    
    # Define os parâmetros da ordem de compra
    ordem_compra = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": ticker,
        "volume": quantidade,
        "type": mt5.ORDER_TYPE_BUY,
        "price": preco,
        "magic": 1,
        "comment": "Trades hackeando a bolsa",
        "type_time": mt5.ORDER_TIME_DAY,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    
    # Envia a ordem de compra
    compra = mt5.order_send(ordem_compra)
    
    # Imprime o resultado da tentativa de envio da ordem
    print(compra)

AttributeError: 'NoneType' object has no attribute 'ask'